# Delta Rule
The Delta Rule is a gradient descent approximator that can be used to find the weight values of a Perceptron. The Delta Rule minimizes an error function (tipically $E=\frac{1}{2}(y - t)^{2}$) by adapting the weight connections in small steps. The step length is defined by the learning rate $\alpha$. In this notebook you will explore the classification capabilities of a single Perceptron by using the delta rule. You will start by setting the connection weights by hand for a simple problem, and then you will apply the delta rule for the same problem, and others.

## Loading the packages

In [1]:
import numpy as np
import matplotlib.pyplot as pl
from ipywidgets import interact, widgets

from matplotlib import animation

## Loading the Perceptron code
In order to the make this nothebook smaller, some of the functions (activation functions, and some of the code allowing the visualization of the results) was implemented in a separate python file. You are free to open it if needed.

<font color="red">**For it to work on Colab, make sure to have uploaded the percpetron.py file in your environment**</font>

In [2]:
import perceptron as pt

## The Dataset
The following script allows you to create a 2D dataset by using the mouse. Pressing 'b' adds points belonging to class A (blue), and pressing 'r' adds points belonging to class B (red). You can create as many points as you desire. The final dataset will contain hence three values per point: x coordinate (-1 ≤ x ≤ 1), y coordinate (-1 ≤ y ≤ 1) and the class ∈ {1,-1}.

In [3]:
!pip install ipympl

<font color="red">**For it to work on Colab, you will need to reload your session (Exécution -> redémarrer la session)**</font>

In [4]:
%matplotlib widget

from google.colab import output
output.enable_custom_widget_manager()

fig = pl.figure(figsize=(6,6))
pl.title("Input Dataset")
pl.xlim((-1.2,1.2))
pl.ylim((-1.2,1.2))

dataset = []

def on_press(event):
    if event.key == 'b':
        dataset.append((event.xdata, event.ydata, -1))
        pl.scatter(event.xdata, event.ydata, color='blue')
        pl.draw()
    elif event.key == 'r':
        dataset.append((event.xdata, event.ydata, 1))
        pl.scatter(event.xdata, event.ydata, color='red')
        pl.draw()

# Attach the event handler
fig.canvas.mpl_connect('key_press_event', on_press);

ModuleNotFoundError: No module named 'google.colab'

This is the dataset you just created. Check that there are no NaNs in the third column. Create once again the dataset if necessary

In [ ]:
dataset

## Finding the weights by hand
In this section you should try to find the set of weights that allows a Perceptron to separate the two classes you previously defined. Use the sliders to modify the value of each one of the connections and the bias of the Perceptron while trying to separate the two classes (blue and red). The curve on the right represents the classification error (MSE). If the modifications you provide improve the classification, you should see the error decreasing.

In [ ]:
%matplotlib inline
plotter = pt.PerceptronPlotter2D(data=np.asarray(dataset))

In [ ]:
fig.clf()
_= interact(plotter.plot_interactive, **plotter.controls)

Notice that Sigmoid outputs value between 0 and 1 and that our classes have been defined as -1 and 1. Is it a problem? Do you have the same problem with hyperbolic tangent?

## The Delta Rule
In the following step we propose to solve the classification problem you defined by using the delta-rule algorithm. Look at the code in compute_delta_w and try to understand it.

$$ neta = (x * w\_x) + (y * w\_y) + b$$
$$ output = f\_act(neta) $$
$$ \Delta w\_x = \alpha * (target - output) * f\_act'(neta) * x $$
$$ \Delta w\_y = \alpha * (target - output) * f\_act'(neta) * y $$
$$ \Delta b = \alpha * (target - output) * f\_act'(neta) $$

In [ ]:
def compute_delta_w(inputs, weights, bias, targets, alpha, activation_function):
    neta = np.dot(inputs, weights) + bias
    output, d_output = activation_function(neta)
    error = targets - output
    d_w_x = alpha * error * d_output * inputs[:,0]
    d_w_y = alpha * error * d_output * inputs[:,1]
    d_b = alpha * error * d_output
    return [d_w_x, d_w_y, d_b]

## Batch learning
When you launch the cell, the weights and the bias are initialized at random, and the algorithm perform some iterations (NUMBER_OF_EPOCHS) doing the following:
+  for each point in the dataset, compute the modifications ( Δw ) to be done at each parameter in order to minimize the error function
+ sum up all the modifications -> batch policy
+ modify the weights and bias of the perceptron

The cell records the effects of the modifications performed in a video. Therefore, you can visualize the learning process afterwards.

In [ ]:
%matplotlib inline

inputs = np.asarray(dataset)[:,0:2]
targets = np.asarray(dataset)[:,2]
weights = np.random.normal(size=2)
bias = np.random.normal(size=1)
activation_function = pt.htan

ALPHA = 0.1
NUMBER_OF_EPOCHS = 30

fig = pl.figure(figsize=(12, 4))
plotter = pt.PerceptronPlotter2D(data=np.asarray(dataset))
plotter.init_animation()

def run_epoch_batch(i, alpha, inputs, weights, bias, targets, activation_function):
    d_w_x, d_w_y, d_b = compute_delta_w(inputs, weights, bias, targets, ALPHA, activation_function)
    weights += np.array([np.sum(d_w_x), np.sum(d_w_y)])
    bias += np.sum(d_b)

    return plotter.data2animation(i, inputs, weights, bias, targets, activation_function)

SHOW_VIDEO = True       # change this flag if you are unable to see the video
if SHOW_VIDEO:
    anim = animation.FuncAnimation(fig, run_epoch_batch, fargs=(ALPHA, inputs, weights, bias, targets, activation_function), frames=NUMBER_OF_EPOCHS, interval=20, blit=True)
    pt.display_animation(anim)
else:
    for i in np.arange(NUMBER_OF_EPOCHS):
        run_epoch_batch(i, ALPHA, inputs, weights, bias, targets, activation_function)

## Exercise
You are free to modify the learning rate (ALPHA) and the number of iterations (NUMBER_OF_EPOCHS).

Try different 2D classification problems and observe the behaviour of the algorithm in terms of:
- Learning rate needed
- Convergence speed
- Oscillations

Bare in mind that, in the current implementation, the parameters (weights and bias) are initialized randomly every time you launch the cell.

Create dataset as shown and perform the following tests:

1. What happens if the boundaries between both classes are well defined?
![separable](https://drive.google.com/uc?id=1jctNojH56KXougUipA8fney25xwFeH8l)

2. What happens if the classes overlap? What could you say about oscillations in the error signal?
![overlapping](https://drive.google.com/uc?id=1pNjh1OQjJgQw_UTEzancv-xEnrKubTfl)

3. What happens if it is not possible to separate the classes with a single line? What could you say about local minima?
![non_separable](https://drive.google.com/uc?id=17QWBDNiw2TuY7EIoH1BoDzDc2YEIm__M)
